# 使用ImageNet預訓練圖像分類模型做影像即時辨識

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## import必要的套件

In [ ]:
import numpy as np
import sys
sys.path.append('/content/drive/MyDrive/2022AI課程/主題課程4-卷積神經網路(CNN)/實作程式/')
import webcam
import time
import torch
import pandas as pd
import torch.nn.functional as F
from torchvision import transforms,models

## 是否有GPU可以使用

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda:0


## 載入預訓練圖像分類模型

In [ ]:
model = torch.load("/content/drive/MyDrive/2022AI課程/影像辨識/fruit30_pytorch_20220814.pth")

In [ ]:
#model = models.resnet18(weights='DEFAULT')
model = model.eval()
model = model.to(device)

## 載入ImageNet 1000圖像分類標籤對應

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/2022AI課程/dataset/ImageNet/index2class.csv")
index2label={}
for idx,row in df.iterrows():
  index2label[row['ID']]=row['Chinese']

In [ ]:
index2label= np.load('/content/drive/MyDrive/2022AI課程/影像辨識/idx_to_labels.npy', allow_pickle=True).item()

In [ ]:
index2label

{0: '哈密瓜',
 1: '奇異果',
 2: '小黃瓜',
 3: '山竹',
 4: '柚子',
 5: '柳橙',
 6: '梨',
 7: '椰子',
 8: '楊梅',
 9: '榴槤',
 10: '檸檬',
 11: '櫻桃',
 12: '火龍果',
 13: '白葡萄',
 14: '石榴',
 15: '砂糖橘',
 16: '紅番茄',
 17: '紅葡萄',
 18: '紅蘋果',
 19: '聖女番茄',
 20: '胡蘿蔔',
 21: '芒果',
 22: '苦瓜',
 23: '草莓',
 24: '荔枝',
 25: '西瓜',
 26: '青蘋果',
 27: '香蕉',
 28: '鳳梨',
 29: '龍眼'}

## 圖像預處理-縮放、裁剪、轉Tensor、歸一化

In [ ]:
test_transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

## 获取摄像头的一帧画面

In [ ]:
webcam.video_stream()   #開始從webcam串流影像
count = 0
Starttime=time.time()
HTML=""         #影片上方要顯示的文字 
while True:
  img = webcam.video_frame(HTML)          #取得目前的影像frame，傳入對應的圖片說明
  if not img:
    break
  img = test_transform(img).unsqueeze(0).to(device) #目前的影像作預處裡
  output = model(img)                 #做前向傳播，得到各分類的分數
  softmax_out = F.softmax(output,dim=1).squeeze()   #對各分類做softmax運算，得到分類機率
  values,indexs = torch.topk(softmax_out,5)     #取可信度最大的5個结果：分類分數，分類索引
  Output=""
  for i in range(len(indexs)):
    Output +="{}({:.1f}%)".format(index2label[int(indexs[i])],values[i].item()*100)
  count+=1
  Stoptime=time.time()
  HTML =str(round(count/(Stoptime-Starttime),2))+"FPS<br>"+Output

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored